In [1]:
import pandas as pd
import re
import nltk
nltk.download('stopwords') 
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [4]:
from bs4 import BeautifulSoup

In [44]:
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...


True

In [5]:
## Load the dataset
df = pd.read_csv('data/all_kindle_review.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [6]:
df[['reviewText','rating']]

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,4
11996,I have read all seven books in this series. Ap...,5
11997,This book really just wasn't my cuppa. The si...,3
11998,"tried to use it to charge my kindle, it didn't...",1


In [8]:
df = df[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [9]:
df.shape

(12000, 2)

In [10]:
##Missing values
print(df.isnull().sum())
print(df['rating'].unique())
print(df['rating'].value_counts())

reviewText    0
rating        0
dtype: int64
[3 5 4 2 1]
rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64


In [11]:
## preprocessing and cleaning
df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [12]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [13]:
## Lower all the cases
df['reviewText'] = df['reviewText'].str.lower() 

In [14]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [38]:
stop_words = set(stopwords.words('english'))

# Step 1: Remove URLs
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://\S+', '', str(x)))

# Step 2: Remove HTML
df['reviewText'] = df['reviewText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())

# Step 3: Remove special characters but keep spaces (THIS IS THE KEY FIX)
# Use \s instead of just spaces to preserve all whitespace
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s-]+', ' ', x))

# Step 4: Normalize spaces BEFORE removing stopwords
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join(x.split()))

# Step 5: Convert to lowercase for stopword removal
df['reviewText'] = df['reviewText'].apply(lambda x: x.lower())

# Step 6: Remove stopwords (now that we have proper spaces)
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))

# Step 7: Final normalization
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join(x.split()))

print(df[['reviewText', 'rating']].head())

                                          reviewText  rating
0  jacerankinmaybeshortbuthesnothingtomesswithast...       1
1  greatshortreadididntwanttoputitdownsoireadital...       1
2  illstartbysayingthisisthefirstoffourbookssoiwa...       1
3  aggieisangelalansburywhocarriespocketbooksinst...       1
4  ididnotexpectthistypeofbooktobeinlibrarywasple...       1


In [39]:
df.head()

,reviewText,rating
0,jacerankinmaybeshortbuthesnothingtomesswithast...,1
1,greatshortreadididntwanttoputitdownsoireadital...,1
2,illstartbysayingthisisthefirstoffourbookssoiwa...,1
3,aggieisangelalansburywhocarriespocketbooksinst...,1
4,ididnotexpectthistypeofbooktobeinlibrarywasple...,1


In [41]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer

In [42]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [45]:

df['reviewText'] = df['reviewText'].apply(lambda x: lemmatize_words(x))

In [47]:
## Train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['reviewText'],df['rating'], test_size=0.2, random_state=42)

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train)
X_test_bow = bow.transform(X_test)

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
X_train_dense = X_train_tfidf.toarray()
X_test_dense = X_test_tfidf.toarray()

In [54]:
X_train_dense

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(9600, 9597))

In [57]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow = GaussianNB().fit(X_train_dense,y_train)
nb_model_tfidf = GaussianNB().fit(X_train_dense,y_train)

In [58]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [60]:
y_pred_bow = nb_model_bow.predict(X_test_dense)
y_pred_bow

array([0, 0, 0, ..., 0, 0, 0], shape=(2400,))

In [ ]:
print(accuracy_score(y_test,y_pred_bow))

0.33458333333333334


In [62]:
confusion_matrix(y_test,y_pred_bow)

array([[ 803,    0],
       [1597,    0]])

In [63]:
y_pred_tfidf = nb_model_tfidf.predict(X_test_dense)
y_pred_tfidf

array([0, 0, 0, ..., 0, 0, 0], shape=(2400,))

In [64]:
accuracy_score(y_test,y_pred_tfidf)

0.33458333333333334